In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PowerTransformer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, roc_auc_score, roc_curve, precision_recall_curve
from sklearn.feature_selection import SelectFromModel
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline
from scipy.stats import randint, uniform
import warnings

In [3]:
path = r"D:\diabetes\diabetes_dataset.csv"
df = pd.read_csv(path)

In [4]:
print(f"Dataset shape: {df.shape}")
print("\nColumns in the dataset:")
print(df.columns.tolist())

Dataset shape: (10000, 21)

Columns in the dataset:
['Unnamed: 0', 'Age', 'Sex', 'Ethnicity', 'BMI', 'Waist_Circumference', 'Fasting_Blood_Glucose', 'HbA1c', 'Blood_Pressure_Systolic', 'Blood_Pressure_Diastolic', 'Cholesterol_Total', 'Cholesterol_HDL', 'Cholesterol_LDL', 'GGT', 'Serum_Urate', 'Physical_Activity_Level', 'Dietary_Intake_Calories', 'Alcohol_Consumption', 'Smoking_Status', 'Family_History_of_Diabetes', 'Previous_Gestational_Diabetes']


In [5]:
df = df.drop('Unnamed: 0', axis=1)

In [6]:
print("\nPerforming feature engineering")
df['Diabetes_Status'] = ((df['HbA1c'] >= 6.5) | (df['Fasting_Blood_Glucose'] >= 126)).astype(int)
print(f"Created target variable 'Diabetes_Status' with distribution:\n{df['Diabetes_Status'].value_counts()}")

df['BMI_Category'] = pd.cut(
    df['BMI'], 
    bins=[0, 18.5, 25, 30, 35, float('inf')],
    labels=['Underweight', 'Normal', 'Overweight', 'Obese', 'Extremely Obese']
)

df['Age_Group'] = pd.cut(
    df['Age'],
    bins=[0, 30, 45, 60, float('inf')],
    labels=['Young', 'Middle_Aged', 'Senior', 'Elderly']
)


Performing feature engineering
Created target variable 'Diabetes_Status' with distribution:
Diabetes_Status
1    9047
0     953
Name: count, dtype: int64


In [8]:
def bp_category(row):
    systolic = row['Blood_Pressure_Systolic']
    diastolic = row['Blood_Pressure_Diastolic']
    
    if systolic < 120 and diastolic < 80:
        return 'Normal'
    elif (systolic >= 120 and systolic < 130) and diastolic < 80:
        return 'Elevated'
    elif (systolic >= 130 and systolic < 140) or (diastolic >= 80 and diastolic < 90):
        return 'Hypertension_Stage1'
    elif systolic >= 140 or diastolic >= 90:
        return 'Hypertension_Stage2'
    else:
        return 'Unknown'

df['BP_Category'] = df.apply(bp_category, axis=1)

df['BMI_Age'] = df['BMI'] * df['Age'] / 100  # Scaled to avoid large values
df['Glucose_BMI'] = df['Fasting_Blood_Glucose'] * df['BMI'] / 100
df['HDL_Total_Ratio'] = df['Cholesterol_HDL'] / df['Cholesterol_Total']
df['LDL_HDL_Ratio'] = df['Cholesterol_LDL'] / df['Cholesterol_HDL']
df['Metabolic_Score'] = (
    (df['BMI'] > 30).astype(int) + 
    (df['Fasting_Blood_Glucose'] > 100).astype(int) +
    (df['Blood_Pressure_Systolic'] > 130).astype(int) +
    (df['Cholesterol_HDL'] < 40).astype(int) +
    (df['Waist_Circumference'] > 100).astype(int)
)
df['Log_Glucose'] = np.log1p(df['Fasting_Blood_Glucose'])
df['Log_HbA1c'] = np.log1p(df['HbA1c'])
df['Log_BMI'] = np.log1p(df['BMI'])
df['Combined_Risk'] = df['Family_History_of_Diabetes'] + (df['Previous_Gestational_Diabetes'])
X = df.drop(['Diabetes_Status', 'HbA1c', 'Fasting_Blood_Glucose'], axis=1)  
y = df['Diabetes_Status']

In [10]:
categorical_cols = ['Sex', 'Ethnicity', 'Physical_Activity_Level', 
                    'Alcohol_Consumption', 'Smoking_Status',
                    'BMI_Category', 'Age_Group', 'BP_Category']
numerical_cols = X.columns.difference(categorical_cols).tolist()
print(f"Using {len(numerical_cols)} numerical features and {len(categorical_cols)} categorical features")
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)
numerical_transformer = Pipeline(steps=[
    ('scaler', StandardScaler()),
    ('power', PowerTransformer(method='yeo-johnson'))
])
categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ]
)

Using 22 numerical features and 8 categorical features


In [11]:
def get_feature_names(column_transformer):
    """Get feature names from all transformers."""
    output_features = []
    for name, pipe, features in column_transformer.transformers_:
        if name == 'num':
            output_features.extend(features)
        elif name == 'cat':
            for feature in features:
                cats = pipe.named_steps['onehot'].categories_[features.index(feature)]
                output_features.extend([f"{feature}_{cat}" for cat in cats])
    
    return output_features

In [13]:
pipeline = ImbPipeline([
    ('preprocessor', preprocessor),
    ('smote', SMOTE(random_state=42)),
    ('feature_selection', SelectFromModel(GradientBoostingClassifier(random_state=42))),
    ('classifier', GradientBoostingClassifier(random_state=42))
])

In [14]:
param_dist = {
    'classifier__n_estimators': randint(100, 500),
    'classifier__learning_rate': uniform(0.01, 0.2),
    'classifier__max_depth': randint(3, 10),
    'classifier__min_samples_split': randint(2, 30),
    'classifier__min_samples_leaf': randint(1, 15),
    'classifier__subsample': uniform(0.6, 0.4),
    'classifier__max_features': ['sqrt', 'log2', None],
    'smote__k_neighbors': randint(3, 10)
}

# Configure the random search
random_search = RandomizedSearchCV(
    pipeline,
    param_distributions=param_dist,
    n_iter=30,
    cv=5,
    scoring='roc_auc',
    n_jobs=-1,
    random_state=42,
    verbose=1
)

# Fit the random search
print("\nPerforming hyperparameter tuning with RandomizedSearchCV...")
print("This may take some time...")
random_search.fit(X_train, y_train)


Performing hyperparameter tuning with RandomizedSearchCV...
This may take some time...
Fitting 5 folds for each of 30 candidates, totalling 150 fits


RandomizedSearchCV(cv=5,
                   estimator=Pipeline(steps=[('preprocessor',
                                              ColumnTransformer(transformers=[('num',
                                                                               Pipeline(steps=[('scaler',
                                                                                                StandardScaler()),
                                                                                               ('power',
                                                                                                PowerTransformer())]),
                                                                               ['Age',
                                                                                'BMI',
                                                                                'BMI_Age',
                                                                                'Blood_Pressure_Diastolic',
                                                                                'Blood_Pressure_Systolic',
                                                                                'Cholesterol_HDL',
                                                                                'Cholesterol_LDL',
                                                                                'Cholesterol_Total',
                                                                                'Combined_Risk',
                                                                                'Dietary_Intake_Calori...
                                        'classifier__n_estimators': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x00000227E6336E10>,
                                        'classifier__subsample': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x00000227E6491640>,
                                        'smote__k_neighbors': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x00000227E6492450>},
                   random_state=42, scoring='roc_auc', verbose=1)

In [15]:
best_pipeline = random_search.best_estimator_
best_params = random_search.best_params_
print("\nBest hyperparameters:")
for param, value in best_params.items():
    print(f"{param}: {value}")
print("\nEvaluating the model on test data")
y_pred = best_pipeline.predict(X_test)
y_pred_proba = best_pipeline.predict_proba(X_test)[:, 1]
print(f"\nAccuracy: {accuracy_score(y_test, y_pred):.4f}")
print(f"AUC: {roc_auc_score(y_test, y_pred_proba):.4f}")

print("\nConfusion Matrix:")
cm = confusion_matrix(y_test, y_pred)
print(cm)

print("\nClassification Report:")
print(classification_report(y_test, y_pred))


Best hyperparameters:
classifier__learning_rate: 0.0849080237694725
classifier__max_depth: 7
classifier__max_features: None
classifier__min_samples_leaf: 11
classifier__min_samples_split: 9
classifier__n_estimators: 288
classifier__subsample: 0.8387400631785948
smote__k_neighbors: 4

Evaluating the model on test data
The history saving thread hit an unexpected error (OperationalError('database or disk is full')).History will not be written to the database.

Accuracy: 1.0000
AUC: 1.0000

Confusion Matrix:
[[ 191    0]
 [   0 1809]]

Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       191
           1       1.00      1.00      1.00      1809

    accuracy                           1.00      2000
   macro avg       1.00      1.00      1.00      2000
weighted avg       1.00      1.00      1.00      2000



In [16]:
plt.figure(figsize=(10, 8))
plt.subplot(2, 1, 1)
fpr, tpr, _ = roc_curve(y_test, y_pred_proba)
plt.plot(fpr, tpr, label=f'AUC = {roc_auc_score(y_test, y_pred_proba):.4f}')
plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend()

plt.subplot(2, 1, 2)
precision, recall, _ = precision_recall_curve(y_test, y_pred_proba)
plt.plot(recall, precision)
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve')

plt.tight_layout()
plt.savefig('model_evaluation.png')
plt.close()
try:
    feature_names = get_feature_names(best_pipeline.named_steps['preprocessor'])
    selector = best_pipeline.named_steps['feature_selection']
    selected_mask = selector.get_support()
    selected_features = [f for f, selected in zip(feature_names, selected_mask) if selected]
    importances = best_pipeline.named_steps['classifier'].feature_importances_

    indices = np.argsort(importances)[-15:] 
    plt.figure(figsize=(12, 8))
    plt.title('Top 15 Feature Importances')
    plt.barh(range(len(indices)), importances[indices], align='center')
    plt.yticks(range(len(indices)), [selected_features[i] for i in indices])
    plt.xlabel('Relative Importance')
    plt.tight_layout()
    plt.savefig('feature_importance.png')
    plt.close()
    
    print("\nTop 5 important features:")
    for i in indices[-5:]:
        print(f"- {selected_features[i]}: {importances[i]:.4f}")
        
except Exception as e:
    print("\nCouldn't extract feature importances due to preprocessing complexity.")
    print(f"Error: {e}")

print("\nModel training and evaluation complete!")
print("Results and visualizations have been saved.")

# Cross-validation to ensure robustness
print("\nPerforming 5-fold cross-validation...")
cv_scores = cross_val_score(best_pipeline, X, y, cv=5, scoring='roc_auc')
print(f"Cross-validation AUC scores: {cv_scores}")
print(f"Mean AUC: {cv_scores.mean():.4f} (±{cv_scores.std():.4f})")

print("\nFull pipeline execution complete!")


Top 5 important features:
- Log_Glucose: 0.2375
- Log_HbA1c: 0.7625

Model training and evaluation complete!
Results and visualizations have been saved.

Performing 5-fold cross-validation...
Cross-validation AUC scores: [1. 1. 1. 1. 1.]
Mean AUC: 1.0000 (±0.0000)

Full pipeline execution complete!
